# Prep OPERA RTC CalVal Slope Comparison, part 2

**Alex Lewandowski; Alaska Satellite Facility, University of Alaska Fairbanks**

## Performs additional data preparation for the OPERA RTC CalVal Slope Comparison Module

**Notebook Requires**
- directory of geotiffs processed with `Prep_OPERA_RTC_CalVal_Slope_Compare_Part_1.ipynb`

**Actions**
- Use the Copernicus global land cover data and HyP3 layover-shadow mask to create a mask of valid land cover pixels unaffected by layover or shadow
- Determine _foreslope_, _backslope_, and _flat_ areas by subtracting the ellipsoidal incidence angles from local incidence angles
- create 3 backscatter geotiffs for each polarity containing valid pixels of the selected ground cover classification in:
    - foreslope regions
    - backslope regions
    - flat regions

In [ ]:
from ipyfilechooser import FileChooser
import math
import numpy as np
from pathlib import Path
import shutil

import numpy.ma as ma
from osgeo import gdal
gdal.UseExceptions()

import opensarlab_lib as osl

## **1. Select the directory holding the output of the Prep_OPERA_RTC_CalVal_Slope_Compare_Part_1.ipynb notebook**

Locate the directory containing the subset local incidence angle map, incidence angle map, layover-shadow mask, dual-pol backscatter geotiffs, and Copernicus land cover data 

```
OPERA_RTC_mosaic_S1*_prepped_for_calval ──
                                         │─  OPERA_RTC_v0.4_vh_S1*_mosaic_clip.tif
                                         │─  OPERA_RTC_v0.4_vv_S1*_mosaic_clip.tif 
                                         │─  OPERA_RTC_v0.4_inc_angle_S1*_mosaic_clip.tif
                                         │─  OPERA_RTC_v0.4_local_inc_angle_S1*_mosaic_clip.tif
                                         │─  OPERA_RTC_v0.4_ls_mask_S1*_mosaic_clip.tif
                                         │─  *_PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-*_clip.tif

```

In [ ]:
print("Select the *_prepped directory output by Prep_OPERA_RTC_CalVal_data_stage1_part1.ipynb")
fc = FileChooser(Path.cwd())
display(fc)

## **2. Warp and Align All Products to Pixel Size of 30**

In [ ]:
data_dir = Path(fc.selected_path)
tiffs = list(data_dir.glob('*.tif'))
resolution = 30
for t in tiffs:
    gdal.Warp(str(t), str(t), xRes=resolution, yRes=resolution, 
              targetAlignedPixels=True, dstNodata=None, copyMetadata=True)   

## **3.  Glob product paths**

In [ ]:
inc_angle = list(data_dir.glob('OPERA_RTC_v0.4_inc_angle_*.tif'))[0]
local_inc_angle = list(data_dir.glob('OPERA_RTC_v0.4_local_inc_angle_*.tif'))[0]
ls = list(data_dir.glob('OPERA_RTC_v0.4_ls_mask_*.tif'))[0]
vh = list(data_dir.glob('OPERA_RTC_v0.4_vh_*.tif'))[0]
vv = list(data_dir.glob('OPERA_RTC_v0.4_vv_*.tif'))[0]
landcover = list(data_dir.glob("*LC100_global*.tif"))[0]

## **4. Mask Unwanted Land Covers and Create GeoTiffs Containing Foreslope, Backslope, and Flat Pixels.**

### Landcover Classifications
https://lcviewer.vito.be/download

- Tree cover
    - 111
    - 112
    - 114
    - 115
    - 116

### Select a ground cover type of interest

In [ ]:
ground_cover = "Tree_Cover"
ground_covers = [0, 111, 112, 113, 114, 115, 116, 121, 122, 123, 124, 125, 126, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200]
valid_covers = [111 , 112, 114, 115, 116]
invalid_covers = [c for c in ground_covers if c not in valid_covers]

ground_cover_dir = data_dir.parent/ground_cover
if not ground_cover_dir.is_dir():
    ground_cover_dir.mkdir()

### Copy incidence angle maps to Tree_Cover directory

In [ ]:
to_deg = [local_inc_angle, inc_angle]
for i, ds in enumerate(to_deg):
    out = ground_cover_dir/f"{ds.stem}_deg.tif"    
    shutil.copy(ds, out)

    if i == 0:
        local_inc_angle = out
    else:
        inc_angle = out

### mask invalid pixels (as determined from layover-shadow mask)

### Create 3 backscatter tiffs for each polarization:

- foreslope backscatter
- backslope backscatter
- flat backscatter 

Remove invalid ground cover and layover/shadow pixels from all output backscatter tiffs 

In [ ]:
valid_landcover = ground_cover_dir/f"{landcover.stem}_valid_{ground_cover}.tif"
shutil.copy(landcover, valid_landcover)

f_landcover = gdal.Open(str(valid_landcover), gdal.GA_Update)
landcover_ar = f_landcover.ReadAsArray()

# Set all invalid ground cover pixels to 0
for val in invalid_covers:
    land_cover_masked = ma.masked_values(landcover_ar, val)
    landcover_ar = land_cover_masked.filled(fill_value=0)

# set all valid ground cover pixels to 1
for val in valid_covers:
    land_cover_masked = ma.masked_values(landcover_ar, val)
    landcover_ar = land_cover_masked.filled(fill_value=1)
    

# set all pixels affected by layover or shadow to 0
f_ls_mask = gdal.Open(str(ls))
ls_array = f_ls_mask.ReadAsArray()

ls_array[ls_array == 0] = 99
ls_mask = ma.masked_where(ls_array==99, np.zeros(ls_array.shape))
ls_array = ls_mask.filled(fill_value=1)   

# Update valid land cover mask with invalid pixels from layover shadow mask
ls_mask = ma.masked_where(ls_array==0, landcover_ar)
landcover_ar = ls_mask.filled(fill_value=0)

# # Save valid land cover mask, containing values=1 for valid land cover pixels unaffected by layover and shadow
f_landcover.GetRasterBand(1).WriteArray(landcover_ar)
f_landcover.FlushCache()

# Subtract elliposidal incidence angles from local incidence angles
local = gdal.Open(str(local_inc_angle))
local_ar = local.ReadAsArray()
ellipse = gdal.Open(str(inc_angle))
ellipse_ar = ellipse.ReadAsArray()
ell_minus_local_inc_arr = local_ar - ellipse_ar


# Create foreslope, backslope, and flat backscatter geotiffs for each polarization
for pol in [vh, vv]:
    foreslope = ground_cover_dir/f"{pol.stem}_foreslope.tif"
    backslope = ground_cover_dir/f"{pol.stem}_backslope.tif"
    flat = ground_cover_dir/f"{pol.stem}_flat.tif"

    slopes = [foreslope, backslope, flat]
    for s in slopes:    
        shutil.copy(pol, s)
        f = gdal.Open(str(s), gdal.GA_Update)
        pol_array = f.ReadAsArray()

        flat_threshold = [-2, 2]

        if "backslope" in str(s):
            mask = ma.masked_where(ell_minus_local_inc_arr<=2, pol_array)
            # Replace values <= upper flat_threshold with 0
            masked_array = mask.filled(fill_value=np.nan)
        elif "foreslope" in str(s):
            # Replace values >= lower flat_threshold with 0
            mask = ma.masked_where(ell_minus_local_inc_arr>=-2, pol_array)
            masked_array = mask.filled(fill_value=np.nan)
        else:
            # Replace values outside of flat_threshold with 0
            mask = ma.masked_where(ell_minus_local_inc_arr>2, pol_array)
            masked_array = mask.filled(fill_value=np.nan)
            mask = ma.masked_where(ell_minus_local_inc_arr<-2, masked_array)
            masked_array = mask.filled(fill_value=np.nan)

        # Remove invalid ground cover and layover/shadow pixels
        invalid_mask = ma.masked_where(landcover_ar==0, masked_array)
        masked_array = invalid_mask.filled(fill_value=np.nan)

        f.GetRasterBand(1).WriteArray(masked_array)
        f.FlushCache()

*Prep_OPERA_RTC_CalVal_Slope_Compare_Part_2 - Version 1.0.0 - August 2023*